In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats
from keras.models import Sequential
from keras.layers import Dense
seed=7
np.random.seed(seed)

Using Theano backend.


In [2]:
def imputeDFColsUsingMedian(dataFrame,cols):
    for col in cols:
        medianOfCol=np.nanmedian(dataFrame[col])
        dataFrame[col].fillna(medianOfCol,inplace=True)
def imputeDFColsUsingMean(dataFrame,cols):
    for col in cols:
        meanOfCol=np.nanmean(dataFrame[col])
        dataFrame[col].fillna(meanOfCol,inplace=True)
def scaleFeature(dataFrame,col):
    maxVal=np.max(dataFrame[col])
    minVal=np.min(dataFrame[col])
    scaledDenom=maxVal-minVal
    dataFrame[col]=(dataFrame[col]-minVal)/scaledDenom
def labelEncodeFeats(dataFrame,listOfFeats):
    for feat in listOfFeats:
        labelEncoder=LabelEncoder()
        encodedFeatValues=labelEncoder.fit_transform(dataFrame[feat])
        dataFrame[feat]=encodedFeatValues
def OneHotEncodeFeats(dataFrame,listOfFeats,ctgrcl_ftrs_msk):
    labelEncodeFeats(dataFrame,listOfFeats)
    oneHotEncoder=OneHotEncoder(categorical_features=ctgrcl_ftrs_msk,sparse=False)
    oneHotEncodedFeats=oneHotEncoder.fit_transform(dataFrame)
    return oneHotEncodedFeats

In [3]:
app_events = pd.read_csv('../Data/app_events.csv')
app_labels = pd.read_csv('../Data/app_labels.csv')
events = pd.read_csv('../Data/events.csv')
events.timestamp=events.timestamp.map(lambda x:pd.Timestamp(x).value)
eventsGrpdByDeviceId=events.groupby('device_id')
gender_age_train = pd.read_csv('../Data/gender_age_train.csv')
gender_age_test = pd.read_csv('../Data/gender_age_test.csv')
label_categories = pd.read_csv('../Data/label_categories.csv')
phone_brand_device_model = pd.read_csv('../Data/phone_brand_device_model.csv',encoding='utf-8')
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id',keep='first')

In [4]:
def joinBrandDeviceModel(deviceIDFrame,brandDeviceModelFrame):
    mergedDF=deviceIDFrame.merge(brandDeviceModelFrame[['device_id','phone_brand','device_model']], 
                                 how='left',on='device_id')
    mergedDF['phone_brand'].fillna('',inplace=True)
    mergedDF['device_model'].fillna('',inplace=True)
    return mergedDF
device_brand_model_trainDF=joinBrandDeviceModel(gender_age_train,phone_brand_device_model)
device_brand_model_testDF=joinBrandDeviceModel(gender_age_test,phone_brand_device_model)

In [5]:
def avgEventDuration(listOfTimeStamps):
    if(len(listOfTimeStamps)<1):
        return 0
    return (np.max(listOfTimeStamps)-np.min(listOfTimeStamps))/len(listOfTimeStamps)
def stdEventDuration(listOfTimeStamps):
    if(len(set(listOfTimeStamps))<=1):
        return 0
    return np.std(listOfTimeStamps)
def avglongChangeFreq(listOfLongs):
    if(len(listOfLongs)<1):
        return 0
    return len(set(listOfLongs))/float(len(listOfLongs))
def avgSqrdlongChangeAmt(listOfLongs):
    if(len(listOfLongs)<1):
        return 0
    return np.sum(np.diff(listOfLongs)**2)/float(len(listOfLongs))
def computeEventBasedFeatures(eventsGrpdByDeviceId):
    eventBasedAggregates=eventsGrpdByDeviceId.aggregate({'timestamp':[np.count_nonzero,avgEventDuration,
                            stdEventDuration],'longitude':[avglongChangeFreq,avgSqrdlongChangeAmt]})
    eventBasedAggregatesFeats=['num_of_evnts','avg_evnt_drtn','std_evnt_drtn','avgLongtdChgFrq','avgSqrdLongtdChgAmt']
    eventBasedAggregates.columns=eventBasedAggregatesFeats
    eventBasedAggregates['device_id']=eventBasedAggregates.index
    return (eventBasedAggregates,eventBasedAggregatesFeats) 

In [6]:
(eventBasedFeaturesDF,eventBasedFeats)=computeEventBasedFeatures(eventsGrpdByDeviceId)

In [7]:
def combineDeviceEventsBrandsFeatures(deviceDF,deviceEventsDF,deviceEventsFeats,deviceBrandsDF):
    device_events_brands =deviceDF.merge(deviceEventsDF, how='left',on='device_id')
    imputeDFColsUsingMean(device_events_brands,deviceEventsFeats)
    device_events_brands=device_events_brands.merge(deviceBrandsDF[['phone_brand','device_model','device_id']], 
                                                  how='left',on='device_id')
    return device_events_brands
deviceEvntsBrnds_trainDF=combineDeviceEventsBrandsFeatures(gender_age_train,eventBasedFeaturesDF,
                                                   eventBasedFeats,device_brand_model_trainDF)
deviceEvntsBrnds_testDF=combineDeviceEventsBrandsFeatures(gender_age_test,eventBasedFeaturesDF,
                                                   eventBasedFeats,device_brand_model_testDF)

In [8]:
def processModelFeatures(contFeats,catFeats,modelDF,categorical_features_mask):
    contFeats=list(set(contFeats))
    catFeats=list(set(catFeats))
    modelFeatures=contFeats.copy()
    modelFeatures.extend(catFeats)
    model_subsetDF=modelDF[modelFeatures]
    for contFeat in contFeats:
        scaleFeature(model_subsetDF,contFeat)
    processedModelFeatures=OneHotEncodeFeats(model_subsetDF,catFeats,categorical_features_mask)
    return processedModelFeatures
catFeats=['phone_brand','device_model']
categorical_features_mask=[False,False,False,False,False,True,True]
modelFeats_trainDF=processModelFeatures(eventBasedFeats,catFeats,deviceEvntsBrnds_trainDF,categorical_features_mask)
modelFeats_testDF=processModelFeatures(eventBasedFeats,catFeats,deviceEvntsBrnds_testDF,categorical_features_mask)

/home/alokkumary2j/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alokkumary2j/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
print(modelFeats_trainDF.shape,modelFeats_testDF.shape)
print(modelFeats_trainDF[11:15,1556:])
deviceEvntsBrnds_trainDF[11:13]

(74645, 1563) (112071, 1658)
[[ 0.          0.          0.00144613  0.10865743  0.42820583  0.09177323
   0.22467937]
 [ 0.          0.          0.01232859  0.02876541  0.23580696  0.08166277
   0.37801029]
 [ 0.          0.          0.00241022  0.          0.09032746  0.04788587
   0.18452801]
 [ 0.          0.          0.01232859  0.02876541  0.23580696  0.08166277
   0.37801029]]


,device_id,gender,age,group,num_of_evnts,avg_evnt_drtn,std_evnt_drtn,avgLongtdChgFrq,avgSqrdLongtdChgAmt,phone_brand,device_model
11,7477216237379271436,F,37,F33-42,7.000000,2.711114e+13,6.637354e+13,0.428571,4084.852943,华为,荣耀6 plus
12,2478205222798310601,F,28,F27-28,52.151315,2.412437e+13,1.116697e+14,0.236296,1081.402878,三星,Galaxy Note 3


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2),min_df=0.0)
device_brand_model_trainDF['brand_model'] = device_brand_model_trainDF['phone_brand'] + ' ' + device_brand_model_trainDF['device_model']
device_brand_model_testDF['brand_model'] = device_brand_model_testDF['phone_brand'] + ' ' + device_brand_model_testDF['device_model']
vect_matrix = vectorizer.fit_transform(device_brand_model_trainDF['brand_model'])
test_vect_matrix = vectorizer.transform(device_brand_model_testDF['brand_model'])

In [11]:
def validateModel(X,y, model):
    kf = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=0)
    for itrain, itest in kf:
        if type(X)==type(pd.DataFrame()):
            train=X.ix[itrain]
            test=X.ix[itest]
        else:
            train = X[itrain,:]
            test = X[itest,:]
        ytrain, ytest = y[itrain], y[itest]
        clf = model.fit(train,ytrain)
        ypred = clf.predict_proba(test)
        print(ypred.shape)
        print(log_loss(ytest, ypred))
        
def getModelOutput(X,y,X2, model):
    kf = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=0)
    for itrain, itest in kf:
        if type(X)==type(pd.DataFrame()):
            train=X.ix[itrain]
            test=X.ix[itest]
        else:
            train = X[itrain,:]
            test = X[itest,:]
        ytrain, ytest = y[itrain], y[itest]
        clf = model.fit(train,ytrain)
        ypred = clf.predict_proba(X2)
        return ypred

In [12]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import LinearSVC
#class CalibModel(object):
class CalibModel(object):
    def __init__(self,clf):
        #clf = MultinomialNB()
        print("Obtained Classifier Instance ",clf)
        self.clf = CalibratedClassifierCV(clf, cv=2, method='sigmoid')
    
    def fit(self, X, y):
        self.clf.fit(X,y)
        return self
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [13]:
#validateModel(vect_matrix, device_brand_model_trainDF['group'], CalibModel(MultinomialNB()))
#validateModel(vect_matrix, phone_brand_master['group'], CalibModel(LinearSVC()))
validateModel(eventsData_train_final,eventsData_train.group,CalibModel(GaussianNB()))
#from sklearn.ensemble import RandomForestClassifier
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(RandomForestClassifier()))
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(DecisionTreeClassifier()))
#from sklearn.ensemble import AdaBoostClassifier
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(AdaBoostClassifier()))

NameError: name 'eventsData_train_final' is not defined

In [ ]:
#from sklearn.ensemble import AdaBoostClassifier
#base_estimator_obj=DecisionTreeClassifier(min_samples_split=10000)
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], 
#              CalibModel(AdaBoostClassifier(base_estimator=base_estimator_obj)))

In [ ]:
#from sklearn.linear_model import LogisticRegression
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(LogisticRegression(C=0.1)))
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(LogisticRegression(C=.2)))
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(LogisticRegression(n_jobs=5,penalty='l1')))
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(LogisticRegression(n_jobs=5,solver='newton-cg',C=0.2)))
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(LogisticRegression(n_jobs=5,solver='newton-cg',C=0.1)))

In [24]:
#Deep Learning : Keras Modelling
# create model
model = Sequential()
model.add(Dense(1600, input_dim=1563, init='uniform', activation='relu'))
model.add(Dense(1563, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

In [25]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
targetLabelEncoder=LabelEncoder()
encodedTarget_train=targetLabelEncoder.fit_transform(device_brand_model_trainDF['group'])
encodedTarget_train

array([10, 10, 10, ...,  6, 10,  7])

In [31]:
# Fit the model
model.fit(modelFeats_trainDF, encodedTarget_train, nb_epoch=10, batch_size=15)

Epoch 1/10
74645/74645 [==============================] - 965s - loss: -86.1852 - acc: 0.0561   
Epoch 2/10
74645/74645 [==============================] - 959s - loss: -86.1852 - acc: 0.0561   
Epoch 3/10
74645/74645 [==============================] - 963s - loss: -86.1852 - acc: 0.0561   
Epoch 4/10
74645/74645 [==============================] - 1051s - loss: -86.1852 - acc: 0.0561  
Epoch 5/10
74645/74645 [==============================] - 1296s - loss: -86.1852 - acc: 0.0561  
Epoch 6/10
74645/74645 [==============================] - 1621s - loss: -86.1852 - acc: 0.0561  
Epoch 7/10
74645/74645 [==============================] - 1676s - loss: -86.1852 - acc: 0.0561  
Epoch 8/10
74645/74645 [==============================] - 1677s - loss: -86.1852 - acc: 0.0561  
Epoch 9/10
74645/74645 [==============================] - 1675s - loss: -86.1852 - acc: 0.0561  
Epoch 10/10
74645/74645 [==============================] - 1674s - loss: -86.1852 - acc: 0.0561  


In [35]:
predictedProbs=model.predict_proba(modelFeats_trainDF, encodedTarget_train)

TypeError: only length-1 arrays can be converted to Python scalars